In [8]:
from ydata_profiling import ProfileReport

from gnn_scheduler import get_project_path
from gnn_scheduler.jssp import (
    load_pickle_instances,
    load_all_from_benchmark,
    get_stat_dataframe,
    CPSolver,
)

In [2]:
# Load instances from "diff_prediction_instances folder"
instances = load_pickle_instances(folder_name="diff_prediction_instances")

Loading instances:   0%|          | 0/49519 [00:00<?, ?it/s]

Loading instances: 100%|██████████| 49519/49519 [00:09<00:00, 5393.66it/s]


In [3]:
df = get_stat_dataframe(instances)
df.head()

,name,n_jobs,n_machines,max_machine_load_and_job_duration,upper_bound,is_optimal,no_solution,difficulty_score
0,naive_generated_instance_35302,20,10,1092,1388,False,False,0.271062
1,naive_generated_instance_49119,10,10,668,791,False,False,0.184132
2,naive_generated_instance_39795,16,10,1005,1232,False,False,0.225871
3,naive_generated_instance_26963,13,10,825,965,False,False,0.169697
4,naive_generated_instance_33786,19,10,1057,1296,False,False,0.226112


In [4]:
# Create report
profile = ProfileReport(df, title="JSSP Difficulty Prediction Instances")
profile.to_file(
    get_project_path()
    / "reports"
    / "jssp_difficulty_prediction_instances.html"
)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/home/pablo/.cache/pypoetry/virtualenvs/gnn-scheduler-rnVoZpeL-py3.11/lib/python3.11/site-packages/ydata_profiling/model/typeset.py:208: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_valid_dtype = pdt.is_categorical_dtype(series) and not pdt.is_bool_dtype(
/home/pablo/.cache/pypoetry/virtualenvs/gnn-scheduler-rnVoZpeL-py3.11/lib/python3.11/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/pablo/.cache/pypoetry/virtualenvs/gnn-scheduler-rnVoZpeL-py3.11/lib/python3.11/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instea

In [11]:
benchmark_instances = load_all_from_benchmark(max_jobs=20, max_machines=10)

benchmark_instances = [
    instance for instance in benchmark_instances if instance.n_machines == 10
]
len(benchmark_instances)

33

In [12]:
# Set new upper bound with CP solver (time_limit=0.1)

for instance in benchmark_instances:
    solution = CPSolver(instance, time_limit=0.1).solve()
    if solution is not None:
        instance.upper_bound = solution["makespan"]
        if solution["status"] == "optimal":
            instance.optimum = solution["makespan"]
            continue
        instance.optimum = None
        continue
    instance.upper_bound = None
    instance.optimum = None

benchmark_df = get_stat_dataframe(benchmark_instances)

In [13]:
benchmark_df

,name,n_jobs,n_machines,max_machine_load_and_job_duration,upper_bound,is_optimal,no_solution,difficulty_score
0,abz5,10,10,868,1267,False,False,0.459677
1,abz6,10,10,742,943,False,False,0.270889
2,ft10,10,10,655,992,False,False,0.514504
3,la16,10,10,717,979,False,False,0.365411
4,la17,10,10,683,784,True,False,0.147877
5,la18,10,10,663,867,False,False,0.307692
6,la19,10,10,685,882,False,False,0.287591
7,la20,10,10,756,910,False,False,0.203704
8,la21,15,10,935,1184,False,False,0.266310
9,la22,15,10,830,989,False,False,0.191566


In [14]:
# Create report
profile = ProfileReport(benchmark_df, title="JSSP Benchmark Instances (10 machines)")
profile.to_file(
    get_project_path() / "reports" / "jssp_benchmark_instances_10_machines.html"
)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/home/pablo/.cache/pypoetry/virtualenvs/gnn-scheduler-rnVoZpeL-py3.11/lib/python3.11/site-packages/ydata_profiling/model/typeset.py:208: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_valid_dtype = pdt.is_categorical_dtype(series) and not pdt.is_bool_dtype(
/home/pablo/.cache/pypoetry/virtualenvs/gnn-scheduler-rnVoZpeL-py3.11/lib/python3.11/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/pablo/.cache/pypoetry/virtualenvs/gnn-scheduler-rnVoZpeL-py3.11/lib/python3.11/site-packages/ydata_profiling/model/typeset.py:208: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instea